## Send MWA data to JSON format for training

In [1]:
import sys
sys.path.append('/users/jmduchar/data/jmduchar/Research/mcgill25/rfi_characterization/python/')
from utils import *

import numpy as np
import yaml
import os
import glob
import json
from astropy.io import fits
import h5py
import matplotlib.pyplot as plt
from mapper import bg_subtract
plt.style.use('seaborn-v0_8')

In [2]:
# Get data
dirpath = '/users/jmduchar/data/jmduchar/Research/mcgill25/ssins_data/'
reference_obs = '1093799552'  # known to contain RFI

with open(dirpath+'gridpoint_dict.yaml', 'r') as file:
    data = yaml.safe_load(file)
    
obsids = get_night(data=data, target_obsid=reference_obs)

In [ ]:
subtracted_data = bg_subtract(data_dir=dirpath+'tars/', obsids=obsids)

channel_width not available in file, computing it from the freq_array spacing.
Antenna metadata are missing for this file. Since this is MWA data, the best way to fill in these metadata is to pass in an mwa_metafits_file which contains information about which antennas were connected when the data were taken. Since that was not passed, the antenna metadata will be filled in from a static csv file containing all the antennas that could have been connected.
Nants_telescope, antenna_names, antenna_numbers, antenna_positions, telescope_location are not set or are being overwritten. Using known values for MWA.
invalid value encountered in multiply
invalid value encountered in divide
channel_width not available in file, computing it from the freq_array spacing.
Antenna metadata are missing for this file. Since this is MWA data, the best way to fill in these metadata is to pass in an mwa_metafits_file which contains information about which antennas were connected when the data were taken. Sinc

Testing 0 emissions.
Number of seeds: 1
Running cycle #1...
Minimum log posterior found for 0 emissions is 9513849.712123465.
Testing 1 emissions.
Number of seeds: 10
Running cycle #1...
Running cycle #2...
Running cycle #3...
Running cycle #4...
Running cycle #5...
Running cycle #6...
Running cycle #7...
Running cycle #8...
Running cycle #9...
Running cycle #10...
Minimum log posterior found for 1 emissions is 7888551.413490874.
Testing 2 emissions.
Number of seeds: 45
Running cycle #1...
Running cycle #2...
Running cycle #3...
Running cycle #4...
Running cycle #5...
Running cycle #6...
Running cycle #7...
Running cycle #8...
Running cycle #9...
Running cycle #10...
Running cycle #11...
Running cycle #12...
Running cycle #13...
Running cycle #14...
Running cycle #15...
Running cycle #16...
Running cycle #17...
Running cycle #18...
Running cycle #19...
Running cycle #20...
Running cycle #21...
Running cycle #22...
Running cycle #23...
Running cycle #24...
Running cycle #25...
Running c

In [ ]:
cb = np.load('../data/coarse_bands_1D.npy')
ssins_list = []

for obsid in obsids:
    file = dirpath + f"tars/{obsid}_SSINS_data.h5"
    
    with h5py.File(file, "r") as hf:
        
        # Access the SSINS
        ssins_rep = hf['Data']['metric_array'][:][:,:,0]  # xx polarization
        
        # Remove CB
        ssins_rep = ssins_rep[:,np.invert(cb)]
        
        # Avg across DTV7
        ssins_rep = np.mean(ssins_rep[:,350:525], axis=1)

        # Remove inf
        ssins_rep = ssins_rep[~np.isinf(ssins_rep)]

        # Append to list
        ssins_list.append(ssins_rep)

In [ ]:
ssins = np.asarray(ssins_list).flatten()

ssins.shape

In [ ]:
# SUBTRACT MEDIAN
ssins = ssins - np.median(ssins)

In [ ]:
plt.figure(figsize=(6,3))
plt.plot(ssins, '.-')
plt.title("Amplitude of SSINS averaged across DTV7 (pointing 0)\nMedian-subtracted", y=1.02)
plt.xlabel("time-step")
plt.ylabel("amplitude (median-subtracted)")
# plt.yscale('log')
# plt.ylim(-1,1)
plt.savefig("./ssins_sample.png",dpi=300,bbox_inches='tight')
plt.show()

In [ ]:
# Structure the data for Stan
stan_data = {
    "N": ssins.shape[0],
    "y": ssins.tolist(),
}

In [ ]:
stan_data

In [ ]:
# Write to JSON
with open("../data/three_state.data.json", "w") as f:
    json.dump(stan_data, f)